# [Project 1] 오디오 데이터 분석 기반 음악 장르 분류

---

## 프로젝트 목표
- 오디오 데이터를 분석하고 딥러딩 모델을 통하여 음악 장르를 예측하는 분류 모델 수행
- 오디오 데이터를 학습 데이터로 사용하기 위해서 사용하는 feature의 종류와 의미에 대해서 학습

---

## 프로젝트 목차

1. **데이터 읽기:** 오디오 데이터를 읽고 feature 정보가 저장된 csv파일을 Dataframe으로 읽기 

2. **오디오 데이터 분석:** 오디오 데이터로부터 활용할 수 있는 feature 종류를 확인

3. **데이터셋 구성:** 오디오 데이터를 feature로 불러와 딥러닝 모델에 사용할 데이터셋으로 구성

4. **딥러닝 모델 학습:** 딥러닝 모델을 구현하고 학습, 평가 및 예측을 수행

---

## 데이터 출처
-  https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification

---

## 프로젝트 개요

한 번쯤 애플 시리(Apple Siri)나 구글 어시스턴트가(Google Assistant) 여러 소리가 섞인 상황에서 내 목소리를 인지하고 답변하는 것을 경험해 보셨을 겁니다. 여러 소리 중에서 어떻게 나의 목소리만 골라내어 인식을 한 것일까요? 

기존에도 신호 처리 기술을 통해 음성이나 오디오 데이터 안에서 원하는 소리를 골라내는 작업을 수행했었습니다. 이후 딥러닝의 발전에 힘입어 이미지 처리처럼 음성 신호 처리에도 딥러닝 모델이 접목되기 시작했고 최근에는 대부분 음성 인식 기술은 딥러닝 기반으로 서비스 되고 있습니다.

이번 프로젝트에서는 간단한 딥러닝 모델과 10가지 장르의 음악 데이터를 가지는 데이터셋을 바탕으로 장르를 분류하는 모델을 구현할 것입니다. 이를 수행하기 위해서 필요한 오디오 데이터 분석 방법 및 feature 추출 방법을 학습하고, 이 feature를 통해 모델을 학습해볼 것입니다.

---

## 1. 데이터 읽기

In [ ]:
# 프로젝트에서 사용할 라이브러리들을 불러옵니다.
!pip install numpy==1.23.5
!pip install librosa==0.9.2
!pip install -U tensorflow==2.10.0

import os
import random

# Tensorflow 관련 디버그 및 경고 메시지 비활성화
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import librosa
import librosa, IPython
import librosa.display as lplt

import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

%matplotlib inline

SEED = 2022
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()
np.random.seed(SEED)
random.seed(SEED)

본 프로젝트에서 사용할 **GTZAN 데이터셋**은 10개 장르의 음악을 각 100개씩 구성하여 총 1000개의 오디오 데이터로 이루어져 있습니다. 각 오디오 파일은 30초의 길이를 가지며 22050 Hz로 샘플링 된 모노 음원입니다.

이번 프로젝트에서는 이 오디오 파일에서 유의미한 feature를 뽑아내어 음악 장르를 분류하는 딥러닝 모델을 만들어볼 것입니다. 그러기 위해 먼저 데이터셋이 어떻게 구성되어 있는지 간략히 살펴보겠습니다.

데이터셋은 플랫폼 상의 `/mnt/data` 디렉토리에 저장되어 있으며 실제 음악 파일은 `/mnt/data/genres_original` 내에 장르별 폴더 안에 `.wav` 파일로 존재합니다 (가령 `genres_original/blues/blues.00000.wav`). 이들을 하나씩 확인해보기 전에 각 파일별로 오디오 데이터에서 뽑아낼 수 있는 대표적인 feature들의 평균과 분산값을 정리한 csv 파일이 있어서 이것부터 확인해보겠습니다.

In [ ]:
root_dir = "/mnt/data"
df = pd.read_csv(os.path.join(root_dir, "features_30_sec.csv"))

df.head()

본 데이터셋에는 `features_30_sec.csv`와 `features_3_sec.csv` 두 종류의 csv 파일이 있는데, 각각 30초 전체에서 feature 값을 계산한 것과, 3초 단위로 잘라서 계산한 것을 의미합니다.

따라서 `features_3_sec.csv`는 하나의 파일 별로 총 10개의 row가 존재하며 `features_30_sec.csv`는 파일 하나에 row 하나만 존재합니다. 그러므로 여기서 읽어온 `features_30_sec.csv`의 전체 row 개수는 1000개가 나올 것입니다.

In [ ]:
df.info()

아래 표는 각각의 feature 의미를 간략하게 정리한 내용입니다. 모델 학습에 사용할 feature의 자세한 설명은 뒤에서 알아보겠습니다.

|feature|의미 |
|:---|:---|
|filename | 음악 파일 이름|
|length | 음악 길이|
|chroma_stft | 프레임당 12음계 크기 표현|
|rms | 프레임당 음악 신호의 에너지량|
|spectral_centroid | 주파수 스펙트럼 중심 |
|spectral_bandwidth | 주파수 스펙트럼 분산|
|rolloff | 전체의 85% 에너지 지점의 스펙트럼 주파수|
|zero_crossing_rate | 주기빈도특성|
|harmony| 배음을 만들어내는 원음 스펙트럼|
|perceptr | 조성 특징|
|tempo | 음악 빠르기|
|mfcc | 멜주파수 캡스트럼 계수|
|label | 음악 장르|

분류해야 할 label 정보가 저장된 `label` column에 대해서 살펴보면 다음과 같습니다.

In [ ]:
df['label'].unique()

## 2. 오디오 데이터 분석

이제 오디오 데이터를 분석하기 위해 사용할 수 있는 feature들을 알아보겠습니다.

위에서 확인해봤던 csv 파일은 그 자체로 feature의 값들을 포함하고 있기에 바로 딥러닝 모델에 적용할 수 있습니다. 하지만 이런 파일이 따로 제공되는 건 극히 드문 일이라 할 수 있습니다.

게다가 이 파일은 오디오 전체 길이에서 구한 feature값의 평균과 분산값처럼 대표값만을 가지고 있습니다. 따라서 이번 프로젝트에서는 아래에서 구할 feature들의 일부를 직접 구해서 딥러닝 모델에 활용할 것입니다.

우선 feature들의 종류를 알아보기 위해 샘플 오디오 파일 하나를 불러오겠습니다. Python에서 오디오 파일을 불러오기 위해 사용하는 라이브러리에는 `librosa`라는 것이 있습니다. 이는 오디오 파일을 numpy array로 불러와 줍니다.

In [ ]:
audio, sr = librosa.load(os.path.join(root_dir, "genres_original", "blues", "blues.00000.wav"))

음악 데이터는 `audio`에 numpy array로 저장되며, `IPython.display.Audio`를 사용하여 재생할 수 있습니다.

`sr`은 sample rate로 아날로그 신호를 디지털로 변환할 때 값을 읽어올 빈도라고 말할 수 있습니다. 오디오에서는 초당 읽어오는 횟수를 주로 나타내며, 따라서 Hz를 통해 표현합니다.

<center><img src="img/sampling.png" width=500></center>

일반적인 음원 CD에는 44,100Hz로 샘플링된 음원이 들어가게 됩니다. 이는 인간의 가청 주파수와 연관된 부분인데, 사람마다 편차는 있지만 인간이 들을 수 있는 소리의 최대 주파수는 20,000Hz 정도라고 합니다. 그런데 음악은 왜 훨씬 높은 주파수인 44,100Hz로 샘플링 되었을까요?

이는 아날로그 신호를 디지털 신호로 바꿀 때 아날로그 신호가 가지는 **최대 주파수의 최소 2배**로 디지털 신호가 샘플링 되었다면 이 디지털 신호를 다시 아날로그 신호로 완벽하게 복원할 수 있다는 **나이키스트 이론(Nyquist Theorem)** 에 따른 것입니다.

그러니까 인간의 가청 주파수가 20,000Hz 정도니 어느 정도 여유 공간을 둔 다음 두배 주파수로 샘플링 하면 나중에 다시 소리를 아날로그로 재생해도 인간이 들을 수 있는 범위의 신호는 모두 완벽하게 복원될 것입니다. 따라서 음원 업계에서는 44,100Hz가 표준처럼 자리잡은 것입니다.

다시 본론으로 돌아와 불러온 음악 데이터를 재생해봅니다.

In [ ]:
# 아래 코드를 실행하고 재생 버튼을 눌러보면 소리가 나올 것입니다.
IPython.display.Audio(audio, rate=sr)

이를 시간 축에서 신호의 폭을 표현하는 waveshow로 출력하면 다음과 같습니다.

waveshow는 일반적으로 음악을 편집하는 환경에서 자주 볼 수 있는 그래프입니다.

In [ ]:
plt.figure(figsize=(15,5))
lplt.waveshow(audio)
plt.show()

이제 데이터를 구분짓는 데 사용할 수 있는 feature를 하나씩 살펴보겠습니다.

### 2.1. Chroma

음악에서 Chroma는 음계를 뜻합니다. 즉 도, 레, 미, 파 같은 것이라고 할 수 있습니다.

이 음계의 조합, 즉 화음은 노래의 조성을 결정하는 요소이고 이는 노래의 느낌에 큰 영향을 준다는 것은 모두 잘 아실 것입니다.

따라서 음악 파일이 가지고 있는 음계의 분포를 확인하면 장르를 분석하는 데 도움을 줄 수도 있을 것입니다.

`librosa` 라이브러리에서는 서양 음악에서 대표적으로 사용하는 12음계 시스템에서 음계 분포를 구하기 위한 `librosa.feature.chroma_stft` 함수를 제공합니다.

In [ ]:
chromagram = librosa.feature.chroma_stft(y=audio, sr=sr, hop_length=10000)
plt.figure(figsize=(16, 6))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=10000, cmap='coolwarm')

전체 30초 길이에서 일정 단위 별로 어떤 음계가 어떻게 분포되었는지 보여주고 있습니다.

세로축이 12개의 음계를 의미하며, 빨간색일수록 해당 시간 프레임에서는 해당 음계가 많이 등장했음을 의미합니다. 

### 2.2 RMS

RMS는 Root Man Square의 약자로 신호 처리 분야에서는 신호가 가지고 있는 에너지를 하나의 수치로 나타낼 때 많이 사용합니다.

엄밀하게는 신호가 가진 에너지는 신호 세기값의 제곱의 합으로 이루어집니다. 오디오 신호에서는 에너지가 높으면 전반적으로 시끄러운 소리라고 할 수 있습니다.

$$ E = \sum_{n}x[n]^{2} $$

RMS는 이를 전체 시간에서 평균을 구한 다음에 제곱근을 씌운 값이라 할 수 있습니다.

$$ RMS = \sqrt{\frac{1}{N}\sum_{n}x[n]^{2}} $$

음악의 장르에 따라서도 이 시끄러운 정도가 달라질 수 있으므로 RMS 값 또한 유의미한 feature로 사용할 수 있을 것입니다.

`librosa`에서는 `librosa.feature.rms` 함수를 사용하면 일정 시간 간격마다 RMS 값의 분포를 구할 수 있습니다.

In [ ]:
rms = librosa.feature.rms(y=audio)

frames = range(len(rms[0]))
t = librosa.frames_to_time(frames)

# 같은 창에서 그래프를 출력하기 위해서 정규화
def normalize(Signal, axis=0):
    return minmax_scale(Signal, axis=axis)

plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio, sr=sr, alpha=0.4)
plt.plot(t, normalize(rms[0]), color='r')

빨간색 선이 RMS 값의 변화입니다.

### 2.3. Spectral Centroid

오디오 신호는 다양한 주파수를 가지는 소리의 조합으로 구성됩니다.

거기에 악기가 다르면 같은 "도"를 연주해도 소리가 다르게 들립니다. 이는 각 악기마다 소리를 낼 때 조합되는 신호 성분이 모두 다르기 때문이라고 할 수 있습니다.

<img src="img/1-1.jpg" width="90%" height="90%" title="스펙트럼" alt="스펙트럼"></img>

위 그림 또한 하나의 피아노 건반을 쳤을 때 나오는 주파수 성분과 그것의 세기를 묘사한 것입니다.

이처럼 소리가 가지고 있는 주파수 성분을 분석해보는 것을 **스펙트럼 분석(Spectral Analysis)** 이라고 합니다.

앞서 음계 분석이 12개의 음계만을 본 것이라면 스펙트럼 분석은 좀더 포괄적으로 주파수 자체의 분포를 보는 것이라고도 할 수 있겠습니다.

그러면 시간에 따라 변화하는 신호에서 주파수 성분은 어떻게 알아볼 수 있을까요? 이를 가능하게 해주는 것이 바로 그 유명한 **푸리에 변환 (Fourier Transform)** 입니다.

<center><img src="img/fourier_transform.gif" width=500></center>

아래 예제는 `scipy`의 함수를 통해 간단하게 푸리에 변환을 수행해봅니다.

In [ ]:
N = len(audio)

yf = scipy.fft.rfft(audio)
xf = scipy.fft.rfftfreq(N, 1 / sr)

plt.figure(figsize=(12, 6))
plt.plot(xf, np.abs(yf))

위 그래프는 예제로 불러온 `blues.00000.wav` 파일의 주파수 성분을 푸리에 변환를 통해 구한 것입니다.

확인해보니 대략 500Hz 부근의 주파수가 가장 많이 존재하고, 전체적으로도 2,000Hz 미만의 주파수가 대부분을 차지하는 것을 확인할 수 있습니다.

그러면 Spectral Centroid라는 것은 무엇일까요?

위의 푸리에 변환은 오디오 신호의 전체 길이에서 주파수 성분을 구한 것입니다. 하지만 음악은 특정 시간대에는 특정 악기 위주로 연주가 이루어지다가 다른 시간대에는 다른 악기들이 주로 연주되는 등 시간대 별로 주파수 특성이 조금씩 달라질 것입니다.

따라서 Spectral Centroid란 전체 오디오 신호를 일정 시간대별로 나누어서 각 시간대별로 **중심(Centroid)이 되는** 주파수를 의미합니다.

Spectral Centroid는 `librosa.feature.spectral_centroid` 함수를 사용하여 구할 수 있습니다.

In [ ]:
spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)

frames = range(len(spectral_centroid[0]))
t = librosa.frames_to_time(frames)

# Plotting the Spectral Centroid along the waveform
plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroid[0]), color='r')

### 2.4. Spectral Rolloff

Spectral rolloff는 스펙트럼 표현에서 얻을 수 있는 또하나의 feature입니다.

낮은 주파수부터 각 주파수의 에너지를 더했을 때 전체 에너지에서 특정 비율까지의 에너지를 포함하는 주파수를 의미합니다.

즉 이 비율이 85%로 설정되었으면 Spectral Rolloff 주파수는 이 이하의 주파수가 가지는 에너지를 모두 더했을 때 전체 에너지의 85%를 가진다는 의미입니다.

Spectral rolloff는 `librosa.feature.spectral_rolloff` 함수를 사용하여 구할 수 있습니다.

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)

frames = range(len(spectral_rolloff[0]))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff[0]), color='r')

### 2.5. Spectral Bandwidth

Spectral Bandwidth는 스펙트럼 표현에서 일종의 분산값 같은 것이라 할 수 있습니다. p차 Spectral Bandwidth은 아래와 같이 계산합니다.

$$ (\sum_{k}S(k)(f(k) - f_{c})^{p})^{\frac{1}{p}} $$

$S(k)$는 주파수 범위를 이산화(discrete) 했을 때 k번째 주파수의 크기(magnitude), $f(k)$는 k번째 주파수, $f_{c}$는 Spectral Centroid를 의미합니다.

$p = 2$인 경우 표준편차 식에서 평균을 구하기 위해 개수로 나눠주는 부분만 없기 때문에 주파수 표현의 가중 표준 편차(Weighted Standard Deviation)라고도 할 수 있습니다.

Spectral Bandwidth는 `librosa.feature.spectral_bandwidth` 함수를 통하여 구할 수 있습니다.

In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y=audio, sr=sr)

frames = range(len(spectral_bandwidth_2[0]))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2[0]), color='r')

### 2.6. Zero-Crossing Rate

Zero-crossing rate란 신호가 양의 값에서 음의 값으로, 또는 음의 값에서 양의 값으로 변화하는 정도를 시간당 비율로 계산한 값입니다.

따라서 이 값은 신호의 급격한 변화가 많으면 증가할 것이므로 음성 인식이나 타악기 소리를 구분할 때 많이 사용하는 feature라고 합니다.

Zero-crossing rate는 `librosa.feature.zero_crossing_rate` 함수를 사용하여 구할 수 있습니다.

In [ ]:
zero_crossings_rate = librosa.feature.zero_crossing_rate(y=audio)

plt.figure(figsize=(14, 5))
plt.plot(zero_crossings_rate[0])

## 3. 데이터셋 구성

이제 실제 오디오 데이터를 모두 불러와서 딥러닝 모델에 학습할 수 있는 형태로 가공하겠습니다.

구체적으로는 다음 과정을 거칠 것입니다.
1. `librosa`를 통해 오디오 데이터 로드 및 feature 추출
2. 학습용과 테스트 데이터를 분리 후 정규화

### 3.1. 오디오 데이터 로드 및 feature 추출

데이터가 저장된 디렉토리의 `genres_original`에서 모든 음원 파일을 하나씩 불러와서 feature를 추출한 다음 numpy array로 구성합니다.

사용할 feature는 다음과 같습니다.
- Spectral Centroid
- Spectral Rolloff
- Spectral Bandwidth
- Zero Crossing Rate

In [ ]:
# 먼저 label로 사용할 이름들을 정수형으로 바꿉니다.
label_id = {
    "blues": 0,
    "classical": 1,
    "country": 2,
    "disco": 3,
    "hiphop": 4,
    "jazz": 5,
    "metal": 6,
    "pop": 7,
    "reggae": 8,
    "rock": 9,
}

In [ ]:
# feature를 추출하기 위한 함수입니다.
# hop_length는 기본 512를 사용하는 걸 권장합니다. 자세한 내용은 librosa 공식 문서를 참고 바랍니다.
# 이 값을 다른 값으로 바꾸면 코드가 제대로 동작하지 않을 수 있습니다.
def get_audio_feature(audio, sr, hop_length=512):
    rms = librosa.feature.rms(y=audio, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr, hop_length=hop_length)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr, hop_length=hop_length)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr, hop_length=hop_length)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio, hop_length=hop_length)

    feature_vec = np.concatenate(
        [
            rms,
            spectral_centroid,
            spectral_rolloff,
            spectral_bandwidth,
            zero_crossing_rate,
        ],
        axis=0
    ) # [feature_num, sequence_num]

    feature_vec = feature_vec[:, :1290]
    
    return feature_vec

In [ ]:
# 처음에 불러온 DataFrame에 파일 이름과 라벨 정보가 있으므로 이를 메타데이터로 활용합니다.
metadata = df

# 각각 feature들의 vector가 저장될 리스트와
# 대응하는 라벨들이 저장될 리스트입니다.
feature_vec_arr = []
label_list = []

# 오디오 데이터를 하나씩 불러와서 feature들의 vector로 구성합니다.
# 최대 5분 정도 걸릴 수 있습니다.
for i in tqdm(range(len(metadata))):
    cur_file = metadata.iloc[i, 0] # ex) blues.00000.wav
    cur_label = cur_file.split(".")[0] # ex) blues

    if cur_file == 'jazz.00054.wav':
        # Kaggle에서 다운받은 데이터셋에서는
        # 이 파일이 깨진 파일로 올라가 있어 정상 파일을 따로 불러옵니다.
        audio, sr = librosa.load("jazz.00054.wav")
    else:
        audio, sr = librosa.load(os.path.join(root_dir, "genres_original", cur_label, cur_file))

    feature_vec = get_audio_feature(audio, sr)
    feature_vec_arr.append(feature_vec)
    label_list.append(label_id[cur_label])

feature_vec_arr = np.stack(feature_vec_arr, axis=0) # [audio_num, feature_num, sequuence_num]
label_list = np.array(label_list)
feature_vec_arr.shape

총 5종류의 feature를 추출했으며, 각 feature별 sequence의 길이는 1290이고 이것이 총 1000개 모여있는 형태로 전체 데이터셋을 구성하였습니다.

### 3.2. 학습, 테스트 데이터 분리 및 정규화

이제 앞서 구성한 전체 데이터셋에서 학습과 테스트 데이터를 분리합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    feature_vec_arr,
    label_list,
    test_size=0.2,
    random_state=SEED
)

다음으로 학습 데이터를 이용해 정규화를 수행합니다.

정규화는 평균과 표준편차를 이용한 표준화로 적용합니다.

In [ ]:
# feature에 해당하는 차원이 두번째 차원이므로, 이를 제외한 나머지 차원에서
# 평균과 표준편차값을 구합니다.
feature_mean = np.mean(X_train, axis=(0, 2), keepdims=True)
feature_stdv = np.std(X_train, axis=(0, 2), keepdims=True)

# 표준화를 수행합니다.
X_train = (X_train - feature_mean) / feature_stdv
X_test = (X_test - feature_mean) / feature_stdv

## 4. 딥러닝 모델 학습

이제 본격적으로 장르 분류를 위한 딥러닝 모델을 학습해보겠습니다.

딥러닝 모델을 직접 사용하기에 앞서 딥러닝 모델의 간단한 역사와 원리를 짚고 넘어가겠습니다.

딥러닝 모델이란 기계학습 모델의 한 종류로서 **인공 신경망(Artificial Neural Network)** 구조를 사용하는 모델들을 통틀어서 말하는 단어입니다.

현대적인 인공 신경망은 1958년에 [Frank Rosenblatt](https://en.wikipedia.org/wiki/Frank_Rosenblatt)이 제안한 퍼셉트론(Perceptron)에서 시작합니다. 이후 단순 퍼셉트론으로는 비선형 문제를 풀 수 없다는 문제 때문에 80년대까지는 거의 사장되었습니다만, 90년대에 현재 Meta AI에 근무하고 있는 [Yann LeCun](https://en.wikipedia.org/wiki/Yann_LeCun)에 의해 현대적인 Convolutional Neural Network(CNN)의 초기 형태가 개발되기도 하였습니다. 현재 이는 [LeNet](https://en.wikipedia.org/wiki/LeNet)이란 이름으로 알려져 있습니다.

**[퍼셉트론]**
<center><img src="img/Perceptrons.jpeg" width=500></center>

**[LeNet]**
<center><img src="img/lenet.png" width=700></center>

하지만 인공 신경망은 당대의 컴퓨팅 자원으로는 너무 많은 연산량을 요구했기 때문에 주류에 오르지는 못했습니다. 인공 신경망이 다시 주목을 받게 된 계기는 2012년에 발표된 [AlexNet](https://en.wikipedia.org/wiki/AlexNet) 덕분입니다.

AlexNet은 [ImageNet](https://www.image-net.org/)이라 불리는 컴퓨터 비전 분야의 유명한 데이터셋을 이용한 대회인 ImageNet Large Scale Visual Recognition Challenge(ILSVRC)에서 2012년 당시 1등을 한 모델입니다. 이게 유명해진 이유는 기존 기계학습 기반의 모델보다 월등한 성능 차이로 1등을 했음은 물론이고, 인공 신경망의 연산을 위해 **그래픽 카드**를 사용하는 것이 매우 효율적이라는 점을 증명했기 때문입니다.

**[AlexNet]**
<center><img src="img/alexnet.png" width=600></center>

그래픽 카드, 보통은 그래픽 카드 내의 연산 장치인 **GPU (Graphics Processing Unit)** 로도 많이 불리는 장비의 발전으로 딥러닝은 기계 학습의 하위 분야임에도 기계 학습보다 훨씬 유명해지게 되었습니다. 따라서 지금은 사실상 인공지능과도 동의어로 많이 알려진 상태입니다.

### 4.1. 딥러닝 모델의 원리

딥러닝 모델, 즉 인공 신경망은 퍼셉트론의 원리에 기반해서 작동합니다. 딥러닝 모델을 구성하는 대표적인 키워드는 아래와 같습니다.
- 퍼셉트론
- 활성화 함수 (Activation Function)
- 역전파 (Back Propagation)

위 키워드들은 **딥러닝 모델**을 얘기할 때만 유효한 것들로서, 저것들이 딥러닝 모델을 다른 기계학습 모델과 구분짓는 큰 특징이라 할 수 있습니다. 

이것들이 각각 하는 역할을 구체적으로 알아보면 좋겠으나, 내용이 너무 길어져서 여기서는 아주 간단하게만 설명하고 넘어가겠습니다. 따라서 많은 설명이 생략될 것이고, 여기의 설명만으로는 이해가 되지 않을 수 있어서 추가 내용은 다른 과목이나 검색을 통해 찾아보시는 걸 권장합니다.

먼저 아래 퍼셉트론을 여러개 중첩 시킨 **다층 퍼셉트론(Multilayer Perceptron)** 의 구조를 확인하겠습니다.
<center><img src="img/multilayer_perceptron.png" height=300></center>

보면 크게 input layer, hidden layers, output layer로 구성된 것을 확인할 수 있습니다. Layer에 존재하는 각 값들은 보통 node라고 지칭합니다. 퍼셉트론 모델은 각 node끼리 연결된 선들에 **학습 가능한 값 (가중치, weights)** 을 두고 이를 input node의 값과 하나씩 곱해서 더한 다음 **활성화 함수**를 적용시켜 이어지는 hidden layer를 이루는 node의 값을 결정합니다. 여기서 각 input 값이 다음 layer의 모든 node와 연결되기 때문에 다층 퍼셉트론을 **Fully-connected layer**라고도 많이 부릅니다.

이것들이 여러층 쌓아서 최종 output layer에서 결과값을 만들면, 우리가 주어진 입력에 원하는 정답값과 비교하기 위하여 **손실 함수(Loss Function)**  에 적용합니다. 그러면 이 손실 함수의 값을 낮추기 위해 **경사 하강법(Gradient Descent)** 을 적용하는데, 이때 기울기(Gradient)를 계산하기 위해 **역전파(Back Propagation)** 알고리즘을 사용합니다.

그러면 계산된 기울기 값을 기존 가중치 값에서 빼주면 하나의 학습 과정이 마무리 됩니다. 이를 데이터셋의 모든 데이터에 완료했으면 하나의 **Epoch**가 완료되고, 이 Epoch를 여러번 반복해서 원하는 성능이 나올 때까지 모델을 학습합니다.

### 4.2. 딥러닝 모델 정의 및 학습

이제 딥러닝 모델을 직접 구현해보겠습니다.

딥러닝 모델을 손쉽게 구현할 수 있는 대표적인 프레임워크로는 **Tensorflow와 PyTorch**가 있습니다. 둘 사이의 장단점이 명확하기에 뭐가 더 낫다를 얘기하는 건 무의미하지만, Tensorflow는 보통 기업에서 딥러닝 모델을 활용한 서비스를 구축할 때 많이 사용하는 편이고, PyTorch는 연구 분야에서 많이 사용하는 편입니다.


여기서는 Tensorflow가 간단하게 모델 정의, 학습, 평가를 해보기에 적합하기 때문에 Tensorflow를 사용하겠습니다.
(PyTorch는 학습을 위한 함수를 따로 만들어야 해서 조금 복잡한 편입니다.)

아래에서 등장하는 코드들은 구체적인 설명은 생략하겠습니다. 주석으로 간략한 설명을 적어두었으므로 자세한 내용은 공식 API 문서를 참고 바랍니다.

In [ ]:
# 모델이 기대하는 입력 데이터의 모양을 지정하기 위해 feature 개수와 sequence 개수를 가져옵니다.
num_features, num_seq = X_train[0].shape

# Sequential은 Tensorflow에서 지원하는 모델 Layer 전용 리스트라고 할 수 있습니다.
model = tf.keras.Sequential([
    # 현재 입력 데이터는 2차원이기 때문에 다층 퍼셉트론에 적용하기 위해 1차원으로 바꿔줍니다.
    tf.keras.layers.Flatten(),

    # 다층 퍼셉트론의 첫번째 Layer를 정의합니다.
    # Fully-connected Layer는 Tensorflow에서 Dense라는 이름을 사용합니다.
    # 첫번째 Layer에서는 input_shape을 지정해주는 것이 바람직합니다.
    # 활성화 함수는 'activation'으로 지정하고 여기서는 제일 흔히 사용하는 ReLU 함수를 사용합니다.
    tf.keras.layers.Dense(256, activation='relu', input_shape=(num_features * num_seq, )),
    # 딥러닝 모델이 과적합 되는 것을 방지하기 위한 Dropout layer를 추가합니다.
    tf.keras.layers.Dropout(0.2),

    # 두번째 Layer를 추가합니다.
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    # 세번째 Layer를 추가합니다.
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    # 마지막 Layer는 분류기(Classifier)로 활용합니다.
    # 현재 우리가 분류해야 하는 것은 10개의 장르이므로 10개의 값을 출력하게 만듭니다.
    # 그리고 각 값을 확률값으로 바꿔주기 위해 Softmax 활성화 함수를 적용합니다.
    tf.keras.layers.Dense(10, activation='softmax'),
])

다음으로 손실 함수 최적화를 위해 Optimizer는 Adam을 사용하며, 손실 함수는 분류 작업이므로 Cross Entropy 함수를 사용합니다.

Tensorflow에서는 label 데이터가 정수로 이루어졌을 때는 `sparse_categorical_crossentropy`를 활용합니다.

In [ ]:
# 모델의 optimizer와 손실 함수를 지정하기 위해 compile 메소드를 사용합니다.
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

epochs와 batch size를 조절하여 학습을 수행합니다.

In [ ]:
num_epochs = 30
batch_size = 32

# 모델 학습을 위해 fit 메소드를 사용합니다.
history = model.fit(
    X_train,
    y_train,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_split=0.2, # 전체 학습 데이터의 20%는 validation에 사용합니다.
)

학습 결과를 출력하는 그래프를 출력합니다.

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

학습 결과가 어떤 것 같나요? 모델이 학습 데이터는 잘 학습한 것으로 보이는데 Validation 데이터의 결과는 전혀 개선되지 않는 것을 확인할 수 있을 것입니다.

따라서 모델이 과적합(Overfitting) 된 것으로 보입니다. 어쩌면 기본적인 모델이 우리의 데이터를 학습하기에 적절하지 않은 모델일 수도 있습니다.

학습 결과만 보면 희망적이진 않지만 과연 테스트 데이터에서는 어떤 결과가 나올까요?

### 4.2. 모델 평가

학습된 모델을 바탕으로 테스트 데이터의 정확도를 출력합니다.

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=-1)

test_accuracy = accuracy_score(y_test, y_pred)
test_f1_score = f1_score(y_test, y_pred, average='macro')

print(f'테스트 정확도: {test_accuracy * 100:.3f}%')
print(f'테스트 F1 Score: {test_f1_score:.5f}')

여전히 좋지 못한 결과가 나오고 있습니다.

추가적으로 혼동 행렬은 어떤 모습을 보이는지 확인해보겠습니다.

In [ ]:
cm = confusion_matrix(y_pred, y_test)

plt.figure(figsize=(12, 12))
sns.heatmap(cm, annot=True)

어떤 것 같나요? 일부 라벨은 잘 맞춘 것을 확인할 수 있지만 다른 클래스에서 오답률이 높아 낮은 정확도가 나오는 것으로 보입니다.

## 제출

제출을 위해 feature 추출까지 이루어진 제출용 테스트 데이터가 `submission_test.npy` 파일로 저장되어 있습니다.

프로젝트에서 사용한 기본 모델을 개선하여 이 데이터의 **정확도가 50% 이상**이 나오도록 해봅시다.

모델을 개선할 수 있는 몇가지 방안을 소개하면 다음과 같습니다.
- 1차원 Convolution layer (Conv1D) 활용
- RNN 계열의 layer (Vanilla RNN, LSTM, GRU) 활용
- Batch Normalization layer 활용

추가로 데이터를 적용하기에 앞서 적절히 정규화를 수행해야 하는 점 참고 바랍니다.

예측한 결과를 아래와 같이 DataFrame으로 구성하고 `submission.csv`로 저장하여 제출합니다. 라벨 값은 본 프로젝트에서 사용한 `label_id`를 그대로 따라갑니다.

| index | label |
|-------|--------|
| 0     | 5  |
| 1     | 1 |
| 2     | 2 |
| 3     | 1 |
| 4     | 0 |

In [ ]:
X_sub = np.load("submission_test.npy")
X_sub.shape

### 채점

결과 csv 파일을 저장 후, 아래 코드를 실행하면 채점을 받을 수 있습니다.

**아래 코드를 수정하면 채점이 불가능 합니다.**

In [ ]:
# 채점을 수행하기 위하여 로그인
import sys
sys.path.append('vendor')
from elice_challenge import check_score, upload

In [ ]:
# 제출 파일 업로드
await upload()

In [ ]:
# 채점 수행
await check_score()

---

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>